### Проверка работоспособности Whisper на исходных данных
Цель - выбрать пару песен и получить их перевод с помощью Whisper, сравнить с настоящим текстом.

In [1]:
import os
import torch
import librosa
from transformers import WhisperProcessor, WhisperForConditionalGeneration

class TextExtractor(torch.nn.Module):
    def __init__(self, device='cuda'):
        super().__init__()
        self.device = device

        # Model params
        self.processor = WhisperProcessor.from_pretrained("openai/whisper-small")
        self.model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

        for param in self.model.parameters():
            param.requires_grad = False

    def forward(self, x) -> torch.Tensor:
        p = self.processor(x, sampling_rate=16000, return_tensors="pt")
        input_features = p.input_features.to(self.device)
        generated_ids = self.model.generate(input_features)
        transcription = self.processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        return transcription

Код для загрузки аудио: загрузка с помощью librosa с последующим приведением двухканальной записи в одноканальную.

In [2]:
def load_audio(audio_filename, dataset_path="V:/dataset/music"):
    waveform, sr = librosa.load(os.path.join(dataset_path, audio_filename), sr=16000)

    if waveform.ndim > 1:
        waveform = waveform.mean(axis=1)  # Convert to mono if stereo

    return waveform, sr

Выполняем скользящий инференс по 30 секунд аудио:

In [7]:
text_extractor = TextExtractor().to('cuda')

def extract_text_from_audio(audio_filename, dataset_path="V:/dataset/music"):
    waveform, sr = load_audio(audio_filename, dataset_path=dataset_path)
    chunk_size = 30 * sr

    for i in range(0, len(waveform), chunk_size):
        chunk = waveform[i:i+chunk_size]

        if len(chunk) < chunk_size // 2:
            continue  # skip too short

        print(text_extractor(chunk))

Получаем корректный текст песни на русском с небольшими неточностями (Крест -> Грест, ко груди -> как груди):

In [8]:
extract_text_from_audio("54/144654.mp3")

 Music
 В дом нежданно ворвалась беда Сына мать как груди прижимает Забирают его навсегда
 Что же ты, сын, натворил? Зачем ты себя погубил? Воплем безумным в груди, Станет сердце несчастное. Не увидишь ли степи широкие? Больше взору они не нужны. Чем прогневал ты власти жестокие?
 Ты, Алеша, сынок, расскажи, Чем прогневал ты власти жестокие? Ты, Алеша, сынок, расскажи, Не читал я речей недозволенных, Не срывался с насиженных мест, Заступился я за обездоленных, Чем и выбрал свой тягостный крест,
 Стуку, стук, колеса стук Поровозный кричит гудок Пласть прощальной зарежет слух
 Плач прощальный печальный звук Вот и скрылся для линьба лоток
 Спасибо.


Аналогичный результат для иностранного языка

In [9]:
extract_text_from_audio("70/25270.mp3")

 Pourquoi se sourire quand tu m'aperçois ? Qu'est-ce qui te fait rire ? Pourquoi se regarde là ?
 Tu sembles déçu par ce que tu vois mais qu'attendais-tu que vous l'études moi ?
 Oh mon miroir, dis-moi si je le prie, oh
 Je vous l'aime et je vous oublie Dans ces trôles de petiria
 du vrai et du beau, impossible chimère, mes rêves sont à l'eau.
 Oui, qu'il reste un espoir Comme moi ton avis Je voudrais tant savoir Oh mon miroir Dis-moi si je le plus fous Je voudrais vivre
 Ma vie de venir quelqu'un de bien Je voudrais mais je m'oublier Dans ces trônes de petits liens
 Yn ymwneud yma, yw'r unrhyw yw'r unrhyw yw'r unrhyw yw'r unrhyw yw'r unrhyw yw'r unrhyw yw'r unrhyw yw'r unrhyw yw'r unrhyw yw'r unrhyw yw'r unrhyw yw'r unrhyw yw'r unrhyw yw'r unrhyw yw'r unrhyw yw'r unrhyw yw'r unrhyw yw'r unrhyw yw'r unrhyw yw'r unrhyw yw'r unrhyw yw'r unrhyw yw'r unrhyw yw'r unrhyw yw'r unrhyw yw'r unrhyw yw'r unrhyw yw'r unrhyw yw'r unrhyw yw'r unrhyw yw'r unrhyw yw'r unrhyw yw'r unrhyw yw'r unrhyw yw